# Natal kicks

Analyze the outcome of a distribution of natal kicks applied to a binary system at a core-collapse stage

The main focus of the study is to understand how the inclination between the pre and post collapse orbital plane
is shifted as a direct consequence of a random component of the momentum kick outside the pre collapse plane.

## load modules

In [ ]:
%load_ext autoreload
%autoreload 2

import pprint
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

import poskiorb

## define some utility functions

In [ ]:
def get_value_from_cc_file(fname: str, string: str) -> Union[str, float]:
    '''return value associated with string

    Parameters
    ----------
    fname : `string`
       Name of file where to search for information.

    string : `string`
       What to look for in `fname`.

    Returns
    -------
    value : `string/float`
       Formatted returned value.
    '''

    with open(fname, 'r') as f:
        lines = f.readlines()

    for line in lines:
        if string in line:
            if string == 'sn_model':
                return line.strip().split()[1]
            else:
                return float(line.strip().split()[1])


def get_cc_info(star_fname: str, binary_fname: str) -> dict:
    '''get all important parameters of star & binary at core-collapse

    Parameters
    ----------
    star_fname : `string`
       Name of file with info of collapsing star.

    binary_fname : `string`
       Name of file with info on binary system at core-collapse.

    Returns
    -------
    CollapseInfo : `dictionary`
       Information of binary system and collapsing star at core-collapse.
    '''

    m1_pre_cc = get_value_from_cc_file(star_fname, 'mass_pre_cc')
    m1_c_core_mass = get_value_from_cc_file(star_fname, 'c_core_mass_pre_cc')
    m1_remnant_mass = get_value_from_cc_file(star_fname, 'remnant_mass')
    m1_fallback_fraction = get_value_from_cc_file(star_fname, 'fallback_fraction')
    m2 = get_value_from_cc_file(binary_fname, 'companion_mass')
    P = get_value_from_cc_file(binary_fname, 'period_pre_cc')

    # store everything in a dictionary
    CollapseInfo = {'m1': m1_pre_cc, 'm1_c_core_mass': m1_c_core_mass,
            'm1_remnant_mass': m1_remnant_mass, 'm1_fallback_fraction': m1_fallback_fraction,
            'm2': m2, 'P': P}

    return CollapseInfo


def get_inclinations_inside_box(x, y, z, weights,
        xlow=None, ylow=None, xhigh=None, yhigh=None, zlow=None, zhigh=None,
        verbose=False):
    '''Get z values for all the (x,y) pairs that falls within the square defined
    by (xlow,ylow),(xhigh,yhigh)

    Parameters
    ----------
    x : `array`
       Values on the xaxis. Either `P_post` or `a_post`

    y : `array`
       Values on the yaxis. Tipically eccentricty (`e_post`)

    z : `array`
       Values on the zaxis. Use for inclination values only (`cosi`)
    
    weights : `array`
        Weights associated to (x,y,z) values

    xlow : `float`
       Lower left x value

    ylow : `float`
       Lower left y value
       
    zlow : `float`
       Lower z value
       
    xhigh : `float`
       Upper right x value

    yhigh : `float`
       Upper right y value
       
    zhigh : `float`
       Upper z value

    verbose : `boolean`
       Whether to output more information to user
    '''
    
    # stack points in a 3D array
    points = np.column_stack((x, y, z))

    if verbose:
        box = f'[({xlow:.2f}, {ylow:.2f}), ({xhigh:.2f}, {yhigh:.2f})]'
        print(f'defining which points are in/out of box: {box}')

    ll = np.array([xmin, ymin, zmin])  # lower-left
    ur = np.array([xmax, ymax, zmax])  # upper-right
    
    # indexes for points inside box
    inidx = np.all(np.logical_and(ll <= points, points <= ur), axis=1)
    
    inbox = points[inidx]
    outbox = points[np.logical_not(inidx)]
    
    if verbose:
        print(f'\tpoints inside box: {len(inbox)} ({len(inbox)/len(points)*100:5.2f} percent)')
        print(f'\tpoints outside box: {len(outbox)} ({len(outbox)/len(points)*100:5.2f} percent)')

    return np.arccos(inbox[:,2])*180/np.pi, weights[inidx]

## get info on collapse stage

In [ ]:
star_fname = '/home/asimazbunzel/Projects/HMXB-NSBH/src/tides/cc_data/star_at_core_collapse.data'
binary_fname = '/home/asimazbunzel/Projects/HMXB-NSBH/src/tides/cc_data/binary_at_core_collapse.data'

Info = get_cc_info(star_fname=star_fname, binary_fname=binary_fname)

In [ ]:
pprint.pprint(Info)

## apply kick distribution to binary using `poskiorb` module

In [ ]:
binary = poskiorb.binary.BinarySystem(m1=Info['m1'],
            m1_core_mass=Info['m1_c_core_mass'],
            m1_remnant_mass=Info['m1_remnant_mass'],
            m1_fallback_fraction=Info['m1_fallback_fraction'],
            m2=Info['m2'], P=Info['P'])

In [ ]:
binary.set_natal_kick_distribution(
    n_trials=100000,
    distribution_id='Maxwell',
    kick_scaling=lambda x: (1-binary.m1_fallback_fraction)*x)
binary.get_natal_kick_distribution()
binary.get_orbital_distribution(verbose=True)
binary.get_post_kick_grid()

## plot distribution of the 3D post-kick orbits (a, e, cosi)

### orbital period vs eccentricity

In [ ]:
plt.style.use('style.mpl')
fig, ax = plt.subplots(figsize=(3,3))

ax.set_xlabel('period post-kick [days]')
ax.set_ylabel('eccentricity post-kick')

ax.set_xscale('log')
ax.set_xlim([4e1, 3e4])
ax.set_ylim([0,1])

ax.scatter(binary.P_post, binary.e_post, s=0.1);

### orbital period vs inclination and eccentricity vs inclination

In [ ]:
plt.style.use('style.mpl')
fig, axs = plt.subplots(figsize=(5,3), ncols=2, sharey=True)

axs[0].set_xlabel('period post-kick [days]')
axs[0].set_ylabel('inclination between pre- & post-kick')
axs[1].set_xlabel('eccentricity post-kick')

axs[0].set_xscale('log')
axs[0].set_xlim([4e1, 3e4])
axs[0].set_ylim([0, 180])
axs[1].set_xlim([0, 1])

axs[0].scatter(binary.P_post, np.arccos(binary.cosi) * 180 / np.pi, s=0.1)
axs[1].scatter(binary.e_post, np.arccos(binary.cosi) * 180 / np.pi, s=0.1);

### colorbar on (P, e) plane

In [ ]:
plt.style.use('style.mpl')
fig, ax = plt.subplots(figsize=(3,3))

ax.set_xlabel('period post-kick [days]')
ax.set_ylabel('eccentricity post-kick')

ax.set_xscale('log')
ax.set_xlim([4e1, 5e3])
ax.set_ylim([0,1])

cb = ax.scatter(binary.P_post, binary.e_post, s=0.05, c=np.arccos(binary.cosi) * 180 / np.pi,
                vmin=0, vmax=180, rasterized=True)

plt.colorbar(cb, label='inclination between pre- & post-kick', ticks=[0, 45, 90, 135, 180]);

# plt.subplots_adjust(left=0.16, bottom=0.16)
# plt.savefig('porb_vs_ecc_zinc.svg')

#### Borders on the (P,e) plane

In [ ]:
print('P borders: ', binary.P_post_borders)
print('e borders: ', binary.e_post_borders)

In [ ]:
print('P grid: ', np.unique(binary.P_post_grid))
print('e grid: ', np.unique(binary.e_post_grid))

In [ ]:
print('chosen values P, e :', 271.86268171, 0.45093588)

In [ ]:
plt.style.use('style.mpl')
fig, ax = plt.subplots(figsize=(3,3))

ax.set_xlabel('period post-kick [days]')
ax.set_ylabel('eccentricity post-kick')

ax.set_xscale('log')
ax.set_xlim([4e1, 5e3])
ax.set_ylim([0,1])

ax.scatter(binary.P_post, binary.e_post, s=0.05, c='gray', alpha=0.4, rasterized=True)

mask = (binary.P_post > 212.1429762) & (binary.P_post < 334.11121516) & \
       (binary.e_post > 0.41) & (binary.e_post < 0.5)
ax.scatter(binary.P_post[mask], binary.e_post[mask], s=0.3, c='C0')

mask = (binary.P_post > 85.5271232) & (binary.P_post < 134.69958598) & \
       (binary.e_post > 0.6) & (binary.e_post < 0.7)
ax.scatter(binary.P_post[mask], binary.e_post[mask], s=0.3, c='C1')

for border in binary.P_post_borders:
    ax.axvline(border, ls='--', color='black', lw=0.7)

for border in binary.e_post_borders:
    ax.axhline(border, ls='--', color='black', lw=0.7)

# plt.subplots_adjust(left=0.16, bottom=0.16)
# plt.savefig('porb_vs_ecc_chosen.svg')

### compute weights for each natal kick value

In [ ]:
weights = np.zeros(len(binary.P_post))
for k in range(len(binary.P_post)):
    w = binary.w_post[k]
    theta = binary.theta_post[k]
    phi = binary.phi_post[k]
    tmp = stats.maxwell.pdf(w, scale=265*(1-binary.m1_fallback_fraction)) 
    tmp *= stats.uniform.pdf(np.cos(theta), loc=-1, scale=2)
    tmp *= stats.uniform.pdf(phi, scale=2*np.pi)
    weights[k] = tmp

## for two regions in the (P, e) plane, grab all inclinations within it to plot a histogram/distribution

These two regions will then be compared using a Kolmogorov-Smirnov test to check if the two distributions are similar

In [ ]:
while True:
    
    # randomly draw the first box
    k1 = np.random.randint(0, len(binary.P_post_borders)-1)

    # set limits for that box
    xmin = binary.P_post_borders[k1]
    xmax = binary.P_post_borders[k1+1]
    ymin = binary.e_post_borders[k1]
    ymax = binary.e_post_borders[k1+1]
    zmin=-2
    zmax=2
    
    # grab inclinations
    inclinations1, weights1 = get_inclinations_inside_box(
        binary.P_post, binary.e_post, binary.cosi, weights,
        xmin, xmax, ymin, ymax, zmin, zmax,
        True)
    
    # exit if have inclinations inside box, else try again
    if len(inclinations1) > 0: break
        
# repeat same logic for the second box
while True:
    
    # randomly draw the first box
    k2 = np.random.randint(0, len(binary.P_post_borders)-1)
    
    while k1 == k2:
        k2 = np.random.randint(0, len(binary.P_post_borders)-1)
        
    # set limits for that box
    xmin = binary.P_post_borders[k2]
    xmax = binary.P_post_borders[k2+1]
    ymin = binary.e_post_borders[k2]
    ymax = binary.e_post_borders[k2+1]
    zmin=-2
    zmax=2
    
    # grab inclinations
    inclinations2, weights2 = get_inclinations_inside_box(
        binary.P_post, binary.e_post, binary.cosi, weights,
        xmin, xmax, ymin, ymax, zmin, zmax,
        True)
    
    # exit if have inclinations inside box, else try again
    if len(inclinations2) > 0: break

### plot distributions

In [ ]:
plt.style.use('style.mpl')
fig, ax = plt.subplots(figsize=(3,3))

ax.set_xlabel('inclination')
ax.set_ylabel('density')

ibins = np.linspace(0,180,10)
hist1, bin_edges1 = np.histogram(inclinations1, bins=ibins, weights=weights1)
hist2, bin_edges2 = np.histogram(inclinations2, bins=ibins, weights=weights2)

# hist1/=np.sum(hist1)
# hist2/=np.sum(hist2)

ax.set_xlim([0,180])
ax.set_ylim([0,max(np.max(hist1), np.max(hist2))])

left,right = bin_edges1[:-1],bin_edges1[1:]
X_Z = np.array([left,right]).T.flatten()
Y_Z = np.array([hist1,hist1]).T.flatten()
ax.fill_between(X_Z, 1e-5*np.ones(len(Y_Z)), Y_Z, label='inclination set 1',
    color='darkslateblue', alpha=0.75, linewidth=2);

left,right = bin_edges2[:-1],bin_edges2[1:]
X_Z = np.array([left,right]).T.flatten()
Y_Z = np.array([hist2,hist2]).T.flatten()
ax.fill_between(X_Z, 1e-5*np.ones(len(Y_Z)), Y_Z, label='inclination set 2',
    color='orange', alpha=0.5, linewidth=2);

ax.legend()

ks = stats.ks_2samp(inclinations1, inclinations2)
ax.annotate('p-value = {:.2f}'.format(ks.pvalue), xy=(0.58,0.62), xycoords='figure fraction', fontsize=9,
           bbox=dict(facecolor='lightgray', edgecolor='dimgray', boxstyle='round, pad=0.4'));

# plt.subplots_adjust(left=0.20, bottom=0.16)
# plt.savefig('hist_inclination.svg')

KS p-value is usually $>> 0.1$, which means that we can safely assume that within each bin in the (P, e) plane,
the distribution of inclinations has a similar shape